Working with data

PyTorch has two primitives to work with data: torch.utils.data.DataLoader and torch.utils.data.Dataset. Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset.

In [4]:
# 在导入语句之后添加设备定义
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [5]:
# 定义设备 - 支持 macOS
device = torch.device(
    "mps" if torch.backends.mps.is_available() else  
    "cuda" if torch.cuda.is_available() else       
    "cpu"                                          
)
print(f"Using device: {device}")

Using device: mps


**注意点：**
学会检查和使用GPU/MPS加速训练
确保数据和模型都在同一设备上（使用 .to(device)）
了解不同硬件平台的兼容性处理

In [6]:
# Download training data from open datasets.
# 使用 FashionMNIST 数据集。每个 TorchVision 都Dataset包含两个参数：transform和target_transform分别用于修改样本和标签
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),# 数据转换
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

Dataset我们将作为参数传递给DataLoader。这将包装一个可迭代对象，用于覆盖我们的数据集，并支持自动批处理、采样、重排和多进程数据加载。
这里我们将批处理大小定义为 64，即 dataloader 可迭代对象中的每个元素将返回一个包含 64 个特征和标签的批处理。

In [7]:
# 创建 DataLoader
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# 查看数据形状
for X, y in test_dataloader: 
    print(f"Shape of X [N,C,H,W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


Shape of X [N,C,H,W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


Creating Models

In [8]:
# 定义模型
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # 创建一个展平层，用于将2D图像数据展平为1D向
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            # 第一个线性层：将784个输入特征（28×28像素）映射到512个神经元
            nn.Linear(28*28, 512),
            nn.ReLU(),
            # 第二个线性层：512到512个神经元
            nn.Linear(512, 512),
            nn.ReLU(),
            # 第三个线性层：512个神经元映射到10个输出（对应FashionMNIST的10个类别）
            nn.Linear(512, 10)
        )
# 前向传播方法
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [9]:
# 实例化模型并移到设备上
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
# 定义损失函数和优化器
loss_fn = nn.CrossEntropyLoss()
# 使用随机梯度下降（SGD）作为优化算法
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)# 设置学习率为0.001

To train a model, we need a loss function and an optimizer.
为了训练模型，我们需要一个损失函数 和一个优化器。
dataloader: 数据加载器，用于批量提供训练数据
model: 神经网络模型
loss_fn: 损失函数，用于计算预测值与真实值之间的误差
optimizer: 优化器，用于更新模型参数

In [11]:
#在单次训练循环中，模型对训练数据集（分批输入）进行预测，并反向传播预测误差以调整模型的参数。
# 训练函数（修复变量命名）
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device) 
        pred = model(x)
        loss = loss_fn(pred, y) # 计算损失
        loss.backward() # 计算梯度
        optimizer.step()# 使用SGD更新参数
        optimizer.zero_grad()# 清零梯度缓存
        if batch % 100 == 0:
            loss_val, current = loss.item(), (batch + 1) * len(x)
            print(f"loss: {loss_val:>7f}  [{current:>5d}/{size:>5d}]")
        

In [12]:
# 测试函数
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)#获取测试数据集的总样本数
    num_batches = len(dataloader)#获取批次数
    model.eval()#将模型设置为评估模式
    test_loss, correct = 0, 0
    #上下文管理器禁用梯度计算，因为在测试阶段不需要计算梯度，这样可以节省内存并加快计算速度
    with torch.no_grad(): 
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)  # 使用小写变量名并移到设备上
            pred = model(x)   # 模型前向传播，得到预测结果
            test_loss += loss_fn(pred, y).item() # 累加损失值
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches # 计算平均损失
    correct /= size  # 计算准确率
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

test_loss /= num_batches：计算所有批次的平均损失
correct /= size：计算整体准确率（正确预测数/总样本数）
##### 总结
这个测试函数的作用是：
在测试数据上评估模型性能
计算并显示模型的准确率和平均损失
通过禁用梯度计算来提高效率
使用模型的评估模式确保测试结果的准确性
在整个训练过程中，每个训练周期（epoch）结束后都会调用此函数来监控模型在测试集上的表现。

In [13]:
# 训练模型
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.317497  [   64/60000]
loss: 2.298547  [ 6464/60000]
loss: 2.283669  [12864/60000]
loss: 2.271442  [19264/60000]
loss: 2.247291  [25664/60000]
loss: 2.235467  [32064/60000]
loss: 2.236711  [38464/60000]
loss: 2.214442  [44864/60000]
loss: 2.206835  [51264/60000]
loss: 2.157861  [57664/60000]
Test Error: 
 Accuracy: 44.3%, Avg loss: 2.163993 

Epoch 2
-------------------------------
loss: 2.183630  [   64/60000]
loss: 2.167983  [ 6464/60000]
loss: 2.116927  [12864/60000]
loss: 2.125722  [19264/60000]
loss: 2.071105  [25664/60000]
loss: 2.028961  [32064/60000]
loss: 2.049572  [38464/60000]
loss: 1.982203  [44864/60000]
loss: 1.980871  [51264/60000]
loss: 1.891472  [57664/60000]
Test Error: 
 Accuracy: 55.4%, Avg loss: 1.903072 

Epoch 3
-------------------------------
loss: 1.942380  [   64/60000]
loss: 1.906816  [ 6464/60000]
loss: 1.797442  [12864/60000]
loss: 1.830434  [19264/60000]
loss: 1.718021  [25664/60000]
loss: 1.677160  [32064/600

Saving Models
A common way to save a model is to serialize the internal state dictionary (containing the model parameters).

In [14]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


Loading Models
The process for loading a model includes re-creating the model structure and loading the state dictionary into it.

In [15]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [16]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
